# Twitter

## Kelly Shaffer, Raphael Nash, Xiaomeng (Vivian) Kong, Michael Muller, Blandon Casenave 

In [125]:
import networkx as nx
import pandas as pd
import urllib
import numpy as np
import matplotlib.pyplot as plt 
import sys
import os
from networkx.algorithms import bipartite as bi
#twitter OAUTH 
import time as tm
import json
import argparse as ag
import tweepy as tw
from tweepy.streaming import StreamListener as sl
from tweepy import OAuthHandler as oa
from tweepy import Stream as st
import oauth2 as oauth
import re
import dateutil.parser as dp
from pytz import timezone as tz
import pytz 

In [126]:
OAUTH_KEYS = {'consumer_key':CONSUMER_KEY, 'consumer_secret':CONSUMER_SECRET,
 'access_token_key':ACCESS_KEY, 'access_token_secret':ACCESS_SECRET}
auth = tw.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [127]:
search = tw.Cursor(api.search, q='#donaldtrump').items(30)

In [128]:
sn = []
text = []
timestamp =[]
for tweet in search:
    print tweet.user.screen_name, tweet.created_at, tweet.text
    timestamp.append(tweet.created_at)
    sn.append(tweet.user.screen_name)
    text.append(tweet.text)

candypie22 2017-07-16 22:11:33 RT @FreeFlowerThink: #notmypresident #donaldtrump #impeachtrump #resist https://t.co/RwBST7qnl4
tarmavis 2017-07-16 22:10:48 RT @MeilleurGeek: Sa popularité en berne, #DonaldTrump replonge dans l’affaire russe https://t.co/10Li7TbPhS https://t.co/frVBXF3AuH
SumiSumishah9 2017-07-16 22:10:25 The #Wahhabi Mind-Set https://t.co/UOA5gMgb1d #Wahhabism #Salafism #SaudiArabia #ISIS #Taliban #BokoHaram #AlShabab #DonaldTrump
carpel_1287 2017-07-16 22:10:25 RT @FreeFlowerThink: #notmypresident #donaldtrump #impeachtrump #resist https://t.co/RwBST7qnl4
darthur111 2017-07-16 22:10:05 RT @BlueSea1964: 🚨One Small North Dakota Town Taught #MarkZuckerberg  Exactly Why #DonaldTrump  Is President -

#MAGA #TrumpNation 

 https…
spooney35 2017-07-16 22:09:23 #DonaldTrump #ImpeachTrump #RussianCollusion #Resist #25thAmendmentNow #MAGA #AltRight @amjoyshow  @cspanwj… https://t.co/49MqidJ4DG
skhall 2017-07-16 22:09:01 For those who are frightened by #DonaldTrump. #amreading #

In [129]:

# Convert lists to dataframe
df = pd.DataFrame()
df['timestamp'] = timestamp
df['sn'] = sn
df['text'] = text

In [130]:
# Prepare ford date filtering. Adding an EST time column since chat hosted by people in that time zone.
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['EST'] = df['timestamp'] - pd.Timedelta(hours=5) #Convert to EST

In [131]:
df['EST'] = pd.to_datetime(df['EST'])

In [132]:
#Subset for the dates required. Can select a specific date or time to examine.
#import time
#df = df[(pd.to_datetime("2017-7-14 20:00:00", format='%Y-%m-%d %H:%M:%S') < df['EST']) & (df['EST'] < pd.to_datetime("2017-7-14 21:00:00", format='%Y-%m-%d %H:%M:%S'))]

In [133]:
# Write out Tweets in case they are needed later.
df.to_csv('620trumptweets.csv',index = False,encoding='utf-8')

In [134]:
# Create a list of the unique usernames in order to see which users we need to retrieve friends for.
allNames = list(df['sn'].unique())

In [137]:
# Initialize dataframe of users that will hold the edge relationships
dfUsers = pd.DataFrame()
dfUsers['userFromName'] =[]
dfUsers['userFromId'] =[]
dfUsers['userToId'] = []
count = 0 

In [ ]:
nameCount = len(allNames)
# The choice to retrieve friends (who the user is following) rather than followers is intentional.
# Either would work. However, many Twitter users follow fewer users than are following them, especially the most popular accounts. 
# This reduces the number of very large calls to Twitter API, which seemed to cause problems.
for name in allNames:
    # Build list of friends    
    currentFriends = []
    for page in tw.Cursor(api.friends_ids, screen_name=name).pages():
        currentFriends.extend(page)
    currentId = api.get_user(screen_name=name).id
    currentId = [currentId] * len(currentFriends)
    currentName = [name] * len(currentFriends)   
    dfTemp = pd.DataFrame()
    dfTemp['userFromName'] = currentName
    dfTemp['userFromId'] = currentId
    dfTemp['userToId'] = currentFriends
    dfUsers = pd.concat([dfUsers,dfTemp])
    time.sleep(5) # avoids hitting Twitter rate limit
    # Progress bar to track approximate progress
    count +=1
    per = round(count*100.0/nameCount,1)
    sys.stdout.write("\rTwitter call %s%% complete." % per)
    sys.stdout.flush() 

Twitter call 120.0% complete.Rate limit reached. Sleeping for: 831


In [140]:
# Again, to limit the number of calls to Twitter API, just do lookups on followers that connect to those in our user group.
# We are not interested in "friends" that are not part of this community.
fromId = dfUsers['userFromId'].unique()
dfChat = dfUsers[dfUsers['userToId'].apply(lambda x: x in fromId)]

In [141]:
# No more Twitter API lookups are necessary. Create a lookup table that we will use to get the verify the userToName
dfLookup = dfChat[['userFromName','userFromId']]
dfLookup = dfLookup.drop_duplicates()
dfLookup.columns = ['userToName','userToId']
#dfCommunity = pd.DataFrame()
#dfCommunity = pd.merge(dfLookup, on='userToId')

In [142]:
dfLookup[:10]

,userToName,userToId
2182,candypie22,111672312.0
1523,RickBasini,545305702.0
2592,lauriedtmann,13352392.0


In [143]:
dfLookup.to_csv('dfLookup.csv',index = False,encoding='utf-8')

## Code Sources

[https://stackoverflow.com/questions/6399978/getting-started-with-twitter-oauth2-python](Getting started with Twitter\OAuth2\Python)

[http://www.techpoweredmath.com/constructing-social-graph-twitter-plotly/#.WWuUx9MrKYW](Constructing a Social Graph With Twitter and Plotly)